# **DATATHON 2023: NTT-DATA CHALLENGE**

## **Requirements**:

## **Main Program**

Read dataset

Some preprocessing

In [45]:
# Drop rows with NaN values
df = df.dropna()

# Split "ORIGEN" into "REGION", "HOSPITAL" and "DEPARTAMENTO"
df['ORIGEN'] = df['ORIGEN'].str.replace('--', '-')
df[['REGION', 'HOSPITAL', 'DEPARTAMENTO']] = df['ORIGEN'].str.split('-', expand=True)
df = df.drop(["ORIGEN"], axis=1)

# Categorical variables
categorical = ['CODIGO', 'PRODUCTO', 'NUMERO', 'REFERENCIA', 'TIPOCOMPRA', 'REGION', 'HOSPITAL', 'DEPARTAMENTO', 'TGL']
df[categorical] = df[categorical].astype('category')

# Numeric variables
numerical_int = ['CANTIDADCOMPRA', 'UNIDADESCONSUMOCONTENIDAS']
numerical_float = ['PRECIO', 'IMPORTELINEA']
df[numerical_float] = df[numerical_float].astype('float')

# Date format
df['FECHAPEDIDO'] = pd.to_datetime(df['FECHAPEDIDO'], format='%d/%m/%y')
df['MES'] = df['FECHAPEDIDO'].dt.month
df['AÑO'] = df['FECHAPEDIDO'].dt.year
df = df.drop('FECHAPEDIDO', axis=1)

# Create new variable
df['PRECIOUNIDAD'] = df['IMPORTELINEA'] / df['CANTIDADCOMPRA']

df.to_csv('../assets/preprocessed_df.csv', index=False)

df.head()

,CODIGO,FECHAPEDIDO,NUMERO,REFERENCIA,CANTIDADCOMPRA,UNIDADESCONSUMOCONTENIDAS,PRECIO,IMPORTELINEA,TIPOCOMPRA,TGL,PRODUCTO,REGION,HOSPITAL,DEPARTAMENTO
0,E99808,2023-01-01,1595724/23,178567.1,60,10,62.590000,375.540000,Compra menor,TRANSITO,APOSITO DE FIBRAS DE POLIACRILATO C/PLATA-3,1,2,60
1,B41691,2016-02-01,72714/16,400403,40,10,102.803729,411.214916,Compra menor,ALMACENABLE,SOLUCION P/ LIMPIEZA Y DESCONTAMINACION DE HER...,0,10,1
2,E64543,2016-02-01,71961/16,403770,20,5,12.100000,48.400000,Compra menor,TRANSITO,APOSITO DE HIDROFIBRA / CINTA-18,0,4,111
3,E65007,2016-02-01,72773/16,20415,100,50,215.325000,430.650000,Concurso,ALMACENABLE,APOSITO DE ESPUMA POLIURETANO / SACRO-11,0,10,1
4,E64911,2017-02-01,86159/17,20701,300,300,792.000000,792.000000,Concurso,ALMACENABLE,APOSITO C/ CARBON Y PLATA-6,0,6,1


New dataset with groupby

In [16]:
new_df = df[['PRODUCTO', 'FECHAPEDIDO', 'TIPOCOMPRA', 'REGION', 'HOSPITAL', 'DEPARTAMENTO', 'TGL', 'CANTIDADCOMPRA', 'UNIDADESCONSUMOCONTENIDAS', 'PRECIO', 'IMPORTELINEA']].copy()

new_df = new_df.groupby(['AÑO', 'MES', 'PRODUCTO', 'HOSPITAL', 'TIPOCOMPRA', 'TGL'], observed=True).agg({'CANTIDADCOMPRA': 'sum', 'UNIDADESCONSUMOCONTENIDAS': 'mean', 'PRECIO': 'mean', 'IMPORTELINEA': 'sum', 'PRECIOUNIDAD': 'mean'}).reset_index()

new_df.drop(['UNIDADESCONSUMOCONTENIDAS', 'PRECIO', 'IMPORTELINEA', 'PRECIOUNIDAD'], axis=1, inplace=True)

new_df.to_csv('../assets/new_df.csv', index=False)

new_df.head()


Split train and test datasets

In [28]:
split_year = 2023
train = new_df.loc[new_df['AÑO'] < split_year]
test = new_df.loc[new_df['AÑO'] >= split_year]

In [29]:
train.to_csv('../assets/train.csv', index=False)
test.to_csv('../assets/test.csv', index=False)